In [60]:
Nmax =8; Nmin = 7;
S = 1/2; I1 = 3/2; I2 = 1/2; 

BN =6479.6773; DN = 5.5440e-3;
gamma = 81.025; 
bBa = 2301; cBa = 75; bFBa = bBa + cBa/3;
#Aparallel = 2376.0; Aperp = 2301.0; bFBa = 1/3 (Aparallel + 2 Aperp); cBa = Aparallel - Aperp;
bF = 60.0; cF =0*8.224; bFF = bF + cF/3;
eq0Q = -117;

In [61]:
import numpy as np
from sympy.physics.wigner import wigner_6j,wigner_9j,wigner_3j,clebsch_gordan
from numpy import linalg as LA

def kdel(x,y):
    if x==y:
        return 1
    else:
        return 0
def reduced(x):
    return np.sqrt(x*(x+1)*(2*x+1))
def nreduced(x,y):
    return np.sqrt((2*x+1)*(2*y+1))

def HN(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(BN*N*(N+1)-DN*N**2*(N+1)**2)

def HNS(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return (gamma*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*reduced(N)*reduced(S)*nreduced(G,Gp)*(-1)**(F1+I1+N+S+1+Gp+Gp)
            *wigner_6j(Np,Gp,F1,G,N,1)*wigner_6j(S,Gp,I1,G,S,1))

def HFBa(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return bFBa/2*kdel(F,Fp)*kdel(F1,F1p)*kdel(N,Np)*kdel(G,Gp)*(G*(G+1)-S*(S+1)-I1*(I1+1))

def HCBa(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0.0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F1+I1+N+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cBa*np.sqrt(30)/3*kdel(F,Fp)*kdel(F1,F1p)*reduced(I1)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*wigner_3j(N,2,Np,0,0,0)*val)

def HQ(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+Jp+F1+I1+S+Jp)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)*
                    wigner_6j(I1,Jp,F1,J,I1,2)*wigner_6j(Np,Jp,S,J,N,2))
    return (eq0Q/4*kdel(F,Fp)*kdel(F1,F1p)*nreduced(G,Gp)*nreduced(N,Np)*
            wigner_3j(N,2,Np,0,0,0)*1/(wigner_3j(I1,2,I1,-I1,0,I1))*val)

def HFF(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    return (bFF*kdel(F,Fp)*kdel(N,Np)*(-1)**(F+I2+N+G+Gp+I1+S)*nreduced(G,Gp)*nreduced(F1,F1p)*reduced(I2)*
            reduced(S)*wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Gp,F1p,N,F1,G,1)*wigner_6j(S,Gp,I1,G,S,1))

def HCF(state):
    (G,N,F1,F,Gp,Np,F1p,Fp)=state
    val=0
    for J in np.arange(np.abs(N-S),N+S+1):
        for Jp in np.arange(np.abs(Np-S),Np+S+1):
            val += ((-1)**(G+Gp+F+I2+I1+1+N+J)*nreduced(J,Jp)**2*wigner_6j(F1,G,N,S,J,I1)*wigner_6j(F1p,Gp,Np,S,Jp,I1)
                    *wigner_6j(I2,F1p,F,F1,I2,1)*wigner_6j(Jp,F1p,I1,F1,J,1)*wigner_9j(J,Jp,1,N,Np,2,S,S,1))
    return (-cF*np.sqrt(30)/3*kdel(F,Fp)*reduced(I2)*reduced(S)*nreduced(G,Gp)*
            nreduced(N,Np)*nreduced(F1,F1p)*wigner_3j(N,2,Np,0,0,0)*val)

def H_noB(state):
    return HN(state)+HNS(state)+HFBa(state)+HCBa(state)+HQ(state)+HFF(state)+HCF(state)

In [62]:
#Construct coupled basis
myStates=[[0,0,0,0]];
for n in np.arange(Nmin,Nmax+1,1):
    for g in np.arange(np.abs(I1-S),I1+S+1):
        for f1 in np.arange(np.abs(n-g),n+g+1):
            for f in np.arange(np.abs(f1-I2),f1+I2+1):
                    myStates=np.append(myStates,[[g,n,f1,f]],axis=0)
myStates=np.delete(myStates,0,axis=0)

In [63]:
#Construct the Hamiltonian
#Hailtonian without field
H_0=np.zeros([len(myStates),len(myStates)])
for m in range(len(myStates)):
    for n in range(m+1):
        a=np.append(myStates[m,:],myStates[n,:])
        H_0[m,n]=H_noB(a)
        if m == n:
            H_0[m,n]=H_0[m,n]/2
H_0=H_0+np.transpose(H_0)        

In [64]:
H=H_0
w, v = LA.eig(H)
N=1
w=np.round(w,decimals=5)
v=np.round(v,decimals=1)
sorted_w=sorted(w)


In [65]:
def matSort(M,index):
    m=len(index)
    C=np.zeros([m,m])
    for i in range(m):
        C[:,i]=M[:,index[i]]
    return C
sort_index = np.argsort(w)
N=1
sorted_w=sorted(w)
sorted_v=matSort(v,sort_index)

In [66]:
#View energy levels with the state composition
for m in range(len(w)):
    n=-m-1
    rr=sorted_v[:,n]
    res = [idx for idx, val in enumerate(rr) if val != 0]
    print(sorted_w[n],end = " ; ")
    for i in res:
        if (i != res[-1]):
            print(rr[i],str(myStates[i,:]),end =" + " )
        else:
            print(rr[i],str(myStates[i,:]))


468588.15182 ; 1.0 [ 2.   8.  10.  10.5]
468557.75451 ; -0.1 [2.  8.  9.  9.5] + 1.0 [ 2.   8.  10.   9.5]
468403.06045 ; -0.1 [1.  8.  9.  9.5] + 1.0 [2.  8.  9.  9.5] + 0.1 [ 2.   8.  10.   9.5]
468384.13388 ; 0.1 [1.  8.  9.  8.5] + 0.1 [2.  8.  8.  8.5] + -1.0 [2.  8.  9.  8.5]
468224.80362 ; -0.1 [1.  8.  8.  8.5] + 1.0 [2.  8.  8.  8.5] + 0.1 [2.  8.  9.  8.5]
468218.93462 ; 0.1 [1.  8.  8.  7.5] + 0.1 [2.  8.  7.  7.5] + -1.0 [2.  8.  8.  7.5]
468057.92615 ; 0.1 [1.  8.  7.  6.5] + 0.1 [2.  8.  6.  6.5] + -1.0 [2.  8.  7.  6.5]
468048.18698 ; 0.1 [1.  8.  7.  7.5] + -1.0 [2.  8.  7.  7.5] + -0.1 [2.  8.  8.  7.5]
467897.963 ; 1.0 [2.  8.  6.  5.5]
467868.50416 ; 1.0 [2.  8.  6.  6.5] + 0.1 [2.  8.  7.  6.5]
463778.08654 ; -1.0 [1.  8.  7.  7.5] + -0.1 [1.  8.  8.  7.5] + -0.1 [2.  8.  7.  7.5]
463767.4212 ; 1.0 [1.  8.  7.  6.5] + 0.1 [2.  8.  7.  6.5]
463576.29519 ; -0.1 [1.  8.  7.  7.5] + 1.0 [1.  8.  8.  7.5] + 0.1 [2.  8.  8.  7.5]
463571.38883 ; 1.0 [1.  8.  8.  8.5] + 0.1

Shorthand notations:
$$
[a] = \sqrt{2a+1}\\
\{a\} = \sqrt{a(a+1)(2a+1)}
$$

Transformation from Hund's case $b_{\beta S}$ to Hund's case $b_{\beta J}$
$$\scriptsize
|SI_1GNF_1I_2F\rangle = \sum_{J=S+N} [JG] (-1)^{G+S+I_1} 
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
|NSJI_1F_1I_2F\rangle
$$

State representations
$$ \scriptsize
\psi = |\eta\Lambda; SI_1GNF_1I_2F\rangle; 
\psi' = |\eta\Lambda; SI_1G'N'F'_1I_2F'\rangle
$$

Matrix elements
$$\scriptsize
\langle\psi|H_N|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'}\delta_{GG'}(B_N N(N+1) - D_N[N(N+1)]^2)
$$

$$\scriptsize
\langle\psi|H_{NS}|\psi'\rangle = \gamma \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'} \{NS\} [GG'] (-1)^{F_1+I_1+N+S+1+G'+G'} \left\{\begin{matrix} N'&G'&F_1\cr G&N&1 \end{matrix}\right\}
\left\{\begin{matrix} S&G'&I_1\cr G&S&1 \end{matrix}\right\}
$$


$$\scriptsize
\langle\psi|H_F^{(Ba)}|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1}\delta_{NN'}\delta_{GG'} \dfrac{b_F^{(Ba)}}{2}  \left(G(G+1)-S(S+1)-I_1(I_1+1)\right) 
$$

$$\scriptsize
\langle\psi|H_c^{(Ba)}|\psi'\rangle = -\delta_{FF'}\delta_{F_1F'_1} c^{(Ba)}\dfrac{\sqrt{30}}{3}\{I_1S\} [GG'NN']
\sum_{J,J'} (-1)^{G+G'+F_1+I_1+N+J'}[JJ']^2
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix} F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix} I_1&J'&F_1\cr J&I_1&1 \end{matrix}\right\}
\left\{\begin{matrix} J&J'&1\cr N&N'&2\cr S&S&1 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_Q|\psi'\rangle = \delta_{FF'}\delta_{F_1F'_1} \dfrac{eq_0Q}{4} [GG'NN']
\left(\begin{matrix} N&2&N'\cr 0&0&0 \end{matrix}\right)
\left(\begin{matrix} I_1&2&I_1\cr -I_1&0&I_1 \end{matrix}\right)^{-1}
\sum_{J,J'} (-1)^{G+G'+J'+F_1+I_1+S+J'}[JJ']^2
\left\{\begin{matrix}F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix}F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix}I_1&J'&F_1\cr J&I_1&2 \end{matrix}\right\}
\left\{\begin{matrix}N'&J'&S\cr J&N&2 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_F^{(F)}|\psi'\rangle = \delta_{FF'}\delta_{NN'} b_F^{(F)} (-1)^{F+I_2+N+G+I_1+S}[GG'F_1F_1'] \{SI_2\}
\left\{\begin{matrix} I_2&F_1'&F\cr F_1&I_2&1 \end{matrix}\right\}
\left\{\begin{matrix}G'&F_1'&N\cr F_1&G&1 \end{matrix}\right\}
\left\{\begin{matrix}S&G'&I_1\cr G&S&1 \end{matrix}\right\}
$$

$$\scriptsize
\langle\psi|H_c^{(F)}|\psi'\rangle = -\delta_{FF'} c^{(F)}\dfrac{\sqrt{30}}{3}\{I_2S\} [GG'NN'F_1F_1']
\left(\begin{matrix} N&2&N'\cr 0&0&0 \end{matrix}\right)
\sum_{J,J'} (-1)^{G+G'+F+I_2+I_1+1+N+J}[JJ']^2
\left\{\begin{matrix} F_1&G&N\cr S&J&I_1 \end{matrix}\right\}
\left\{\begin{matrix} F_1'&G'&N'\cr S&J'&I_1 \end{matrix}\right\}
\left\{\begin{matrix} I_2&F_1'&F\cr F_1&I_2&1 \end{matrix}\right\}
\left\{\begin{matrix} J'&F_1'&I_1\cr F_1&J&1 \end{matrix}\right\}
\left\{\begin{matrix} J&J'&1\cr N&N'&2\cr S&S&1 \end{matrix}\right\}
$$

In [67]:
ind=np.argmax(np.abs(sorted_v),axis=0)
(m,n)=myStates.shape
toWrite=np.zeros((m,n+1))
for i in range(m):
    toWrite[i,:]=np.append(myStates[ind[i],:],sorted_w[i])
np.savetxt("D:/UChicagoWork/Theory/Reproducing_ScO_MW_data/BaF_groundState_MWTransitions_N5-10.csv", toWrite, delimiter=',',fmt='%i %i %i %1.1f %1.4f', header='G,N,F1,F,Energy')

In [68]:
fname="D:/UChicagoWork/Theory/Reproducing_ScO_MW_data/BaF_groundState_MWTransitions_N5-10.csv"
A=np.genfromtxt(fname,  delimiter=' ', skip_header=0)

In [69]:
def energy_difference(Gi,Ni,F1i,Fi,Gf,Nf,F1f,Ff):
    Ei=0;Ef=0
    for i in range(len(A)):
        if (A[i,0]==Gi and A[i,1] == Ni and A[i,2] == F1i and A[i,3] == Fi):
            Ei=A[i,4]
            continue
    for f in range(len(A)):
        if (A[f,0] == Gf and A[f,1] == Nf and A[f,2] == F1f and A[f,3] == Ff):
            Ef=A[f,4]
            continue
    return np.round((Ef-Ei),decimals=3)            

In [70]:
def columnMatSort(M,index):
    m,n=M.shape
    C=np.zeros([m,n])
    for i in range(m):
        C[i,:]=M[index[i],:]
    return C
#List possible transitions
#Selection rule dN = +1,-1, dF=0,+1,-1
Nf=8;Ni=7
initialF=np.arange(4.5,9.5+1)
finalF=np.arange(5.5,10.5+1)
finalF1=np.arange(6,10+1)
initialF1=np.arange(5,9+1)
print("N = ",Ni," -> "," N = ",Nf)
toDisplay=np.array([[0,0,0,0,0,0,0]])
for G in np.arange(np.abs(S-I1),S+I1+1,1):
    for f1i in initialF1:
        for f1f in finalF1:
            for f in finalF:
                for i in initialF:
                    if np.abs(f-i)<= 1 and np.abs(f1f-f1i)<=1:
                        toDisplay=np.append(toDisplay,np.array([[G,f1i,i,G,f1f,f,np.abs(energy_difference(G,Ni,f1i,i,G,Nf,f1f,f))]]),axis=0)
                        #print("G = ",int(G), " F = ",i," -> ","G = ",int(G)," F = ",f,": ", np.abs(energy_difference(G,Ni,f1i,i,G,Nf,f1f,f)))
toDisplay=np.delete(toDisplay,0,axis=0)
toDisplay=columnMatSort(toDisplay,np.argsort(toDisplay[:,6]))
for i in range(len(toDisplay)):
    print("G = ",int(toDisplay[i,0])," F1 = ",int(toDisplay[i,1]), " F = ",toDisplay[i,2]," -> ","G = ",int(toDisplay[i,3])," F1 = ",int(toDisplay[i,4])," F = ",toDisplay[i,5],": ", toDisplay[i,6])

N =  7  ->   N =  8
G =  1  F1 =  5  F =  4.5  ->  G =  1  F1 =  6  F =  5.5 :  0.0
G =  1  F1 =  9  F =  9.5  ->  G =  1  F1 =  10  F =  8.5 :  0.0
G =  1  F1 =  9  F =  8.5  ->  G =  1  F1 =  10  F =  9.5 :  0.0
G =  1  F1 =  9  F =  9.5  ->  G =  1  F1 =  10  F =  9.5 :  0.0
G =  2  F1 =  5  F =  6.5  ->  G =  2  F1 =  6  F =  7.5 :  0.0
G =  2  F1 =  5  F =  7.5  ->  G =  2  F1 =  6  F =  7.5 :  0.0
G =  2  F1 =  5  F =  8.5  ->  G =  2  F1 =  6  F =  7.5 :  0.0
G =  2  F1 =  5  F =  7.5  ->  G =  2  F1 =  6  F =  8.5 :  0.0
G =  2  F1 =  5  F =  8.5  ->  G =  2  F1 =  6  F =  8.5 :  0.0
G =  2  F1 =  5  F =  9.5  ->  G =  2  F1 =  6  F =  8.5 :  0.0
G =  2  F1 =  5  F =  8.5  ->  G =  2  F1 =  6  F =  9.5 :  0.0
G =  2  F1 =  5  F =  9.5  ->  G =  2  F1 =  6  F =  9.5 :  0.0
G =  2  F1 =  5  F =  9.5  ->  G =  2  F1 =  6  F =  10.5 :  0.0
G =  2  F1 =  6  F =  7.5  ->  G =  2  F1 =  6  F =  7.5 :  0.0
G =  2  F1 =  6  F =  8.5  ->  G =  2  F1 =  6  F =  7.5 :  0.0
G =  2  F1 =  6 

G =  1  F1 =  7  F =  4.5  ->  G =  1  F1 =  8  F =  5.5 :  0.0
G =  1  F1 =  7  F =  5.5  ->  G =  1  F1 =  8  F =  5.5 :  0.0
G =  1  F1 =  7  F =  5.5  ->  G =  1  F1 =  8  F =  6.5 :  0.0
G =  1  F1 =  6  F =  8.5  ->  G =  1  F1 =  7  F =  8.5 :  0.0
G =  1  F1 =  7  F =  8.5  ->  G =  1  F1 =  8  F =  9.5 :  0.0
G =  1  F1 =  6  F =  7.5  ->  G =  1  F1 =  7  F =  8.5 :  0.0
G =  1  F1 =  6  F =  9.5  ->  G =  1  F1 =  6  F =  10.5 :  0.0
G =  1  F1 =  5  F =  5.5  ->  G =  1  F1 =  6  F =  5.5 :  0.0
G =  1  F1 =  5  F =  6.5  ->  G =  1  F1 =  6  F =  5.5 :  0.0
G =  1  F1 =  5  F =  5.5  ->  G =  1  F1 =  6  F =  6.5 :  0.0
G =  1  F1 =  5  F =  6.5  ->  G =  1  F1 =  6  F =  6.5 :  0.0
G =  1  F1 =  5  F =  7.5  ->  G =  1  F1 =  6  F =  6.5 :  0.0
G =  1  F1 =  5  F =  6.5  ->  G =  1  F1 =  6  F =  7.5 :  0.0
G =  1  F1 =  5  F =  7.5  ->  G =  1  F1 =  6  F =  7.5 :  0.0
G =  1  F1 =  5  F =  8.5  ->  G =  1  F1 =  6  F =  7.5 :  0.0
G =  1  F1 =  5  F =  7.5  ->  G =  1  

G =  1  F1 =  6  F =  6.5  ->  G =  1  F1 =  6  F =  7.5 :  360099.809
G =  1  F1 =  6  F =  6.5  ->  G =  1  F1 =  6  F =  5.5 :  360099.809
G =  2  F1 =  6  F =  6.5  ->  G =  2  F1 =  7  F =  5.5 :  364399.674
G =  2  F1 =  6  F =  6.5  ->  G =  2  F1 =  6  F =  7.5 :  364399.674
G =  2  F1 =  6  F =  5.5  ->  G =  2  F1 =  7  F =  5.5 :  364409.697
G =  2  F1 =  7  F =  6.5  ->  G =  2  F1 =  8  F =  5.5 :  364549.891
G =  2  F1 =  7  F =  6.5  ->  G =  2  F1 =  6  F =  7.5 :  364549.891
G =  2  F1 =  7  F =  6.5  ->  G =  2  F1 =  8  F =  6.5 :  364549.891
G =  2  F1 =  7  F =  6.5  ->  G =  2  F1 =  7  F =  5.5 :  364549.891
G =  2  F1 =  7  F =  7.5  ->  G =  2  F1 =  8  F =  6.5 :  364555.451
G =  2  F1 =  7  F =  7.5  ->  G =  2  F1 =  6  F =  8.5 :  364555.451
G =  2  F1 =  7  F =  7.5  ->  G =  2  F1 =  6  F =  7.5 :  364555.451
G =  2  F1 =  7  F =  7.5  ->  G =  2  F1 =  7  F =  8.5 :  364555.451
G =  2  F1 =  8  F =  7.5  ->  G =  2  F1 =  9  F =  6.5 :  364696.879
G =  2

In [71]:
G=2
Ni=7
Nf=8
f1i=5
i=5.5
f1f=6
f=6.5
np.abs(energy_difference(G,Ni,f1i,i,G,Nf,f1f,f))

103623.01

In [72]:
A

array([[1.00000000e+00, 7.00000000e+00, 8.00000000e+00, 8.50000000e+00,
        3.59782145e+05],
       [1.00000000e+00, 7.00000000e+00, 8.00000000e+00, 7.50000000e+00,
        3.59799767e+05],
       [1.00000000e+00, 7.00000000e+00, 7.00000000e+00, 7.50000000e+00,
        3.59913825e+05],
       [1.00000000e+00, 7.00000000e+00, 7.00000000e+00, 6.50000000e+00,
        3.59918279e+05],
       [1.00000000e+00, 7.00000000e+00, 6.00000000e+00, 5.50000000e+00,
        3.60088728e+05],
       [1.00000000e+00, 7.00000000e+00, 6.00000000e+00, 6.50000000e+00,
        3.60099809e+05],
       [2.00000000e+00, 7.00000000e+00, 5.00000000e+00, 5.50000000e+00,
        3.64245494e+05],
       [2.00000000e+00, 7.00000000e+00, 5.00000000e+00, 4.50000000e+00,
        3.64274866e+05],
       [2.00000000e+00, 7.00000000e+00, 6.00000000e+00, 6.50000000e+00,
        3.64399674e+05],
       [2.00000000e+00, 7.00000000e+00, 6.00000000e+00, 5.50000000e+00,
        3.64409697e+05],
       [2.00000000e+00, 7.0000